In [ ]:
!pip install xgboost
!pip install imblearn
!pip install wandb
!pip install lightgbm

     |████████████████████████████████| 193.7 MB 53 kB/s 
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 199 kB 19.8 MB/s 
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 1.8 MB 32.7 MB/s 
     |████████████████████████████████| 280 kB 75.8 MB/s 
     |████████████████████████████████| 181 kB 75.7 MB/s 
     |████████████████████████████████| 144 kB 57.2 MB/s 
     |████████████████████████████████| 63 kB 3.5 MB/s 
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21493 sha256=51e6adc94f33cd7507326346e9c9f1f5d79ec9418654a400246bbf085a7dd704
  Stored in directory: /root/.cache/pip/wheels/29/93/c6/762e359f8cb6a5b69c72235d798804cae523bbe41c2aa8333d
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8784 sha256=14ade8bb792873d88d413f5a52bdd4fdfc8f2d93035c530d6

In [ ]:
import pandas as pd
# import torch
# from torch import nn
import datetime
import numpy as np
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV , KFold, train_test_split
from sklearn.model_selection import KFold
import seaborn as sb
import matplotlib.pyplot as plt
from collections import Counter
from imblearn.over_sampling import SMOTE
import wandb
import pickle
from utils_v2 import *
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams




# Load Preprocessed data from utils.py

In [ ]:
data_df=load_processed_csv(train=True)
data_df

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature63,feature64,feature65,feature66,feature67,feature68,feature69,feature70,feature71,label
0,0.302245,0.0,0.2,0.0,1.0,1.0,0.0,0.0,0.0,0.125,...,0,0,2,1,0,0,1,0,0,3
1,-0.224298,0.0,0.3,0.0,1.0,1.0,1.0,0.0,0.0,0.125,...,0,0,1,1,4,5,0,0,0,3
2,0.206510,0.0,0.7,0.0,1.0,1.0,0.0,0.0,0.0,0.000,...,0,0,1,1,4,0,0,0,0,3
3,0.206510,0.0,0.4,0.0,1.0,1.0,1.0,0.0,0.0,0.250,...,0,0,1,1,0,0,0,0,0,3
4,0.206510,0.0,0.4,0.0,1.0,1.0,1.0,0.0,0.0,0.250,...,0,0,1,1,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9552,-0.750840,0.0,0.5,0.0,1.0,1.0,0.0,0.0,0.0,0.250,...,5,1,2,2,0,1,0,0,3,1
9553,-0.750840,0.0,0.5,0.0,1.0,1.0,0.0,0.0,0.0,0.250,...,5,1,2,2,0,1,0,0,3,1
9554,-0.750840,0.0,0.5,0.0,1.0,1.0,0.0,0.0,0.0,0.250,...,5,1,2,2,0,1,0,0,3,1
9555,-0.750840,0.0,0.5,0.0,1.0,1.0,0.0,0.0,0.0,0.250,...,5,1,2,2,0,1,0,0,3,1


In [ ]:
data_df.head()

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature63,feature64,feature65,feature66,feature67,feature68,feature69,feature70,feature71,label
0,0.302245,0.0,0.2,0.0,1.0,1.0,0.0,0.0,0.0,0.125,...,0,0,2,1,0,0,1,0,0,3
1,-0.224298,0.0,0.3,0.0,1.0,1.0,1.0,0.0,0.0,0.125,...,0,0,1,1,4,5,0,0,0,3
2,0.206510,0.0,0.7,0.0,1.0,1.0,0.0,0.0,0.0,0.000,...,0,0,1,1,4,0,0,0,0,3
3,0.206510,0.0,0.4,0.0,1.0,1.0,1.0,0.0,0.0,0.250,...,0,0,1,1,0,0,0,0,0,3
4,0.206510,0.0,0.4,0.0,1.0,1.0,1.0,0.0,0.0,0.250,...,0,0,1,1,0,0,0,0,0,3


# Define KFolder and light_bgm Function

In [ ]:
def xgboost_train(X_train,y_train,X_test,y_test,params=None):
    train_data = xgb.DMatrix(X_train, y_train)
    validation_data = xgb.DMatrix(X_test, y_test)
    if params is None:
#         params = {
# 'boosting_type': 'dart',
# 'objective':'multi:softmax',
# 'num_class':9,
# 'max_depth':7, #Max_depth: The maximum depth of a tree.
# 'min_child_weight':20, #Min_child_weight: The minimum sum of instance weight needed in a child. Keeping it high prevents the child from being too specific and thus helps to avoid overfitting.
# 'gamma':1,#Minimum loss reduction required to make a further partition on a leaf node of the tree. Again, the larger the game, the less likely the model will overfit.
# 'subsample':0.8,#The ratio of rows that are randomly selected prior to growing trees. Subsample can also be used to avoid overfitting
# 'colsample_bytree':0.7,
# 'tree_method':'hist',
# 'eval_metric':'mlogloss',
# 'eta':0.04,#The learning rate. Keeping it high makes the model learn fast but increases the chance of overfitting at the same time
# 'alpha': 1,#L1 regularization term
# 'verbose': 2
# }
        params = {'max_depth':35, 'eta':0.15, 'silent':0, 'objective':'multi:softmax', 'min_child_weight': 2, 'num_class': 4, 'gamma': 2.5, 'colsample_bylevel': 1, 'subsample': 0.95, 'colsample_bytree': 0.85, 'reg_lambda': 0.35 }
    clf = xgb.train(params, train_data , 50, evals=[(train_data,'eval'),(validation_data, 'eval')], verbose_eval=True)
    return clf

def k_fold(k,X,Y,shuffle=False,random_state=None,params=None):
    X,Y=np.array(X),np.array(Y)
    kf = KFold(n_splits=k,random_state=random_state, shuffle=shuffle)
    for train_index, test_index in kf.split(X):
        X_train = X[train_index]
        y_train = Y[train_index]
        X_test = X[test_index]
        y_test = Y[test_index]
        model = xgboost_train(X_train,y_train,X_test,y_test,params)
        yield (model,test_index)
# X = data_df.iloc[:, :-1]
# Y = data_df["label"]
# k_fold(3,X,Y)

# Model Evaluation 

In [ ]:
X = data_df.iloc[:, :-1]
Y = data_df["label"]
print("original distribution:",Counter(Y))

original distribution: Counter({3: 5996, 1: 1597, 2: 1209, 0: 755})


In [ ]:
#upsamlping 
smo = SMOTE(random_state=42)
X_smo, Y_smo = smo.fit_resample(X, Y)
print("smote distribution:",Counter(Y_smo))
X,Y=X_smo, Y_smo#using smote dataset

smote distribution: Counter({3: 5996, 1: 5996, 2: 5996, 0: 5996})


In [ ]:
#wandb.login(key="2e9513866272d50377f6a772b6b310ea16f8bb8f")
#wandb.init(project="my-test-project", entity="kero0000")
# for depth in range(2,15):
# params={ 'boosting_type': 'dart',
# 'objective':'multi:softmax',
# 'num_class':9,
# 'max_depth':14, #Max_depth: The maximum depth of a tree.
# 'min_child_weight':20, #Min_child_weight: The minimum sum of instance weight needed in a child. Keeping it high prevents the child from being too specific and thus helps to avoid overfitting.
# 'gamma':1,#Minimum loss reduction required to make a further partition on a leaf node of the tree. Again, the larger the game, the less likely the model will overfit.
# 'subsample':0.8,#The ratio of rows that are randomly selected prior to growing trees. Subsample can also be used to avoid overfitting
# 'colsample_bytree':0.7,
# 'tree_method':'hist',
# 'eval_metric':'mlogloss',
# 'eta':0.04,#The learning rate. Keeping it high makes the model learn fast but increases the chance of overfitting at the same time
# 'alpha': 1,#L1 regularization term
# 'verbose': 2}
# # wandb.config.update(params)
params = {'max_depth':35, 'eta':0.1, 'silent':0, 'objective':'multi:softmax', 'min_child_weight': 1, 'num_class': 4, 'gamma': 2.0, 'colsample_bylevel': 0.9, 'subsample': 0.84, 'colsample_bytree': 0.88, 'reg_lambda': 0.75 }
params = {'max_depth':35, 'eta':0.15, 'silent':0, 'objective':'multi:softmax', 'min_child_weight': 2, 'num_class': 4, 'gamma': 2.5, 'colsample_bylevel': 1, 'subsample': 0.95, 'colsample_bytree': 0.85, 'reg_lambda': 0.35 }
macro_F1_metric=[]
for model,test_index in k_fold(10,X,Y,shuffle=True,random_state=1,params=params):#k_fold(k,X,Y,shuffle=False,random_state=None)

    # X_train, X_test, y_train, y_test = train_test_split(X, Y,shuffle = "True", test_size =0.999)
    # y_test_pred = model.predict(X_test).argmax(axis=1)
    X_test = np.array(X)[test_index]
    y_test = np.array(Y)[test_index]
    X_test_Dmax = xgb.DMatrix(X_test)
    y_test_pred=model.predict(X_test_Dmax)

    score = accuracy_score(y_test, y_test_pred)
    print("Accuracy score is " + str(score))
    report=classification_report(y_test, y_test_pred)
    macro_F1_metric.append(eval(report.split("\n")[8].split()[4]))
    with open('XGBoost_data/xgb_model_{}.pkl'.format(macro_F1_metric[-1]), 'wb') as f:
        pickle.dump(model, f)#store as pickle
    print('model saved to XGBoost_data/xgb_model_{}.pkl'.format(macro_F1_metric[-1]))
    print(report)

    # Build the plot
#         cm = confusion_matrix(y_test, y_test_pred)
#         plt.figure(figsize=(16,7))
#         sb.set(font_scale=1.4)
#         sb.heatmap(cm, annot=True, annot_kws={'size':18},cmap=plt.cm.Greens, fmt=".0f")

#         plt.xlabel('Predicted label')
#         plt.ylabel('True label')
#         plt.title('Confusion Matrix for Random Forest Model')
#         plt.show()
#    wandb.log({"avg macro_F1_metric": sum(macro_F1_metric)/len(macro_F1_metric)})
# print("avg macro_F1_metric:",sum(macro_F1_metric)/len(macro_F1_metric),depth)

[03:22:34] WARNING: ../src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	eval-mlogloss:1.17100
[1]	eval-mlogloss:1.00897
[2]	eval-mlogloss:0.87894
[3]	eval-mlogloss:0.77813
[4]	eval-mlogloss:0.69094
[5]	eval-mlogloss:0.61761
[6]	eval-mlogloss:0.55800
[7]	eval-mlogloss:0.50567
[8]	eval-mlogloss:0.46273
[9]	eval-mlogloss:0.42614
[10]	eval-mlogloss:0.39383
[11]	eval-mlogloss:0.36555
[12]	eval-mlogloss:0.34223
[13]	eval-mlogloss:0.32226
[14]	eval-mlogloss:0.30440
[15]	eval-mlogloss:0.29164
[16]	eval-mlogloss:0.27939
[17]	eval-mlogloss:0.26715
[18]	eval-mlogloss:0.25626
[19]	eval-mlogloss:0.24895
[20]	eval-mlogloss:0.24195
[21]	eval-mlogloss:0.23535
[22]	eval-mlogloss:0.22771
[23]	eval-mlogloss:0.22077
[24]	

KernelInterrupted: Execution interrupted by the Jupyter kernel.

# Generate a csv model

In [ ]:
result=[]
for i in range(1,5):
    hybrid_df=pd.read_csv(f'/work/{i}.csv')
    result.append(hybrid_df['Target'])

In [ ]:
def generate_sumbit_csv_XGBoost(model,text=""):
    '''auto tranform label 0-3 to 1-4'''
    test_processed_df=load_processed_csv(train=False)
    X = test_processed_df
    X_DM = xgb.DMatrix(X)
    y_test_pred=model.predict(X_DM)
    y_test_pred +=1
    test_df=pd.read_csv('data/test.csv')
    test_df.insert(len(test_df.columns),'Target',y_test_pred)
    submit_df=pd.concat([test_df['Id'],test_df['Target']],axis=1)
    submit_df.to_csv(f'data/test_submit_{text}_{str(datetime.datetime.now())}.csv')
    print('save to '+f'data/test_submit_{text}_{str(datetime.datetime.now())}.csv')
    print('distribution:')
    for i in range(1,5):
        print(i,len(y_test_pred[y_test_pred==i]))
    return True

def generate_sumbit_csv_combine(model_xgb,model_lgbm,model_rf,text="",method='discrete'):
    '''auto tranform label 0-3 to 1-4'''
    test_processed_df=load_processed_csv(train=False)
    X = test_processed_df
    X_DM = xgb.DMatrix(X)
    y_test_pred_1=model_xgb.predict(X_DM)
    y_test_pred_2=model_lgbm.predict(X)
    y_test_pred_3=model_rf.predict(X)
    y_test_pred=[]
    for i in range(len(y_test_pred_1)):
        if method=='discrete':
            # xgb_result,lgbm_result,rf_result=int(y_test_pred_1[i]+1),int(np.argmax(y_test_pred_2[i])+1),int(y_test_pred_3[i]+1)
            xgb_result,lgbm_result,rf_result=result[0][i],result[1][i],result[2][i]
            if lgbm_result==rf_result:#two voters
                y_test_pred.append(lgbm_result)
            else:
                y_test_pred.append(xgb_result)
        else:
            # y_test_pred.append((y_test_pred_1[i]+np.argmax(y_test_pred_2[i])+int(y_test_pred_3[i])+(hybrid_df.iloc[i][0]-1))//3+1)
            # y_test_pred.append((result[2][i]+result[3][i])//2)
            y_test_pred.append((result[2][i])//1)
        # print(xgb_result,lgbm_result,rf_result)

    y_test_pred=np.array(y_test_pred)
    test_df=pd.read_csv('data/test.csv')
    test_df.insert(len(test_df.columns),'Target',y_test_pred)
    submit_df=pd.concat([test_df['Id'],test_df['Target']],axis=1)
    submit_df.to_csv(f'data/test_submit_{text}_{str(datetime.datetime.now())}.csv')
    print('save to '+f'data/test_submit_{text}_{str(datetime.datetime.now())}.csv')
    print('distribution:')
    for i in range(1,5):
        print(i,len(y_test_pred[y_test_pred==i]))
    return True

In [ ]:
with open(('XGBoost_data/xgb_model_0.87.pkl'), 'rb+') as f:
    model_xgb = pickle.load(f)#load model
with open(('/work/RF_data/RF_model_0.98.pkl'), 'rb+') as f:
    model_rf = pickle.load(f)#load modelx
with open(('/work/LightGBM_data/LightGBM_model_0.98.pkl'), 'rb+') as f:
    model_lgbm = pickle.load(f)#load model
generate_sumbit_csv_combine(model_xgb,model_lgbm,model_rf,text="combine",method='average')

[05:26:07] WARNING: ../src/learner.cc:1040: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[05:26:07] WARNING: ../src/learner.cc:749: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
[05:26:07] WARNING: ../src/learner.cc:438: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details a

True

In [ ]:
with open(('XGBoost_data/xgb_model_0.87.pkl'), 'rb+') as f:
    model = pickle.load(f)#load model
    
generate_sumbit_csv_XGBoost(model,'XGBoost')

[03:48:48] WARNING: ../src/learner.cc:1040: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[03:48:48] WARNING: ../src/learner.cc:749: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
[03:48:48] WARNING: ../src/learner.cc:438: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details a

True

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f92fbbcf-fa92-4caa-ad57-5cd0b8c2766a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>